In [9]:
!python -m pip install -U sentence-transformers
!python -m pip install texthero
!python -m pip install zeroshot_topics

  Using cached texthero-1.1.0-py3-none-any.whl (24 kB)
  Using cached pandas-1.4.0-cp38-cp38-win_amd64.whl (10.6 MB)
  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\bishw\Downloads\complaint_data.csv")

df.head()

df1 = df.loc[:,['Complaint ID','Consumer complaint narrative']].copy(deep = True)

df1.head()

df1.dropna(inplace = True)



In [2]:
import texthero as hero
import nltk
nltk.download('punkt')


dfsample = df1.sample(5000)

from nltk.tokenize import sent_tokenize
s = dfsample["Consumer complaint narrative"].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()


s = pd.DataFrame(s, columns = ['Broken_Survey'])
s.head()

s['Broken_Survey'] = hero.clean(s['Broken_Survey'])






[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bishw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = s.Broken_Survey.to_list()
sentence_embeddings = model.encode(sentences, show_progress_bar=True)

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Batches:   0%|          | 0/1435 [00:00<?, ?it/s]

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [4]:
from sentence_transformers import util
import time

print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(sentence_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))



#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])

cluster_map = {}
for i, cluster in enumerate(clusters):
    sent_list = []
    for sentence_id in cluster:
        sent_list.append(sentences[sentence_id])
    cluster_map[i] = sent_list


cluster_map_1 = {}
for k,v in cluster_map.items():
  s = '.'.join(v)
  cluster_map_1[k] = s

cluster_map_1

Start clustering
Clustering done after 15.39 sec

Cluster 1, #406 Elements 
	 
	 
	 
	 ...
	 
	 
	 

Cluster 2, #307 Elements 
	 xxxx xxxx
	 xxxx xxxx
	 xxxx xxxx
	 ...
	 back xxxx xxxx
	 xxxx xxxx years
	 xxxx thanks

Cluster 3, #134 Elements 
	 please help
	 please help
	 please help
	 ...
	 thank help
	 thank help
	 thank help

Cluster 4, #122 Elements 
	 thank
	 thank
	 thank
	 ...
	 thanks
	 thanks
	 thanks

Cluster 5, #118 Elements 
	 account incorrect reporting credit reports xxxx xxxx xxxx xxxx xxxx xxxx
	 xxxx xxxx xxxx account number xxxx reporting erroneous collection credit report
	 account reported xxxx credit report affecting credit
	 ...
	 dispute placed credit report
	 please help removing credit destroyed erroneous report
	 credit bureau reporting inaccurate information report

Cluster 6, #95 Elements 
	 payment xxxx xxxx xxxx made payment xxxx
	 payment made xxxx xxxx xxxx
	 paid xxxx payment xxxx xxxx
	 ...
	 still confused credit got xxxx payment xxxx payment
	 paid

{0: '.....................................................................................................................................................................................................................................................................................................................................................................................................................',
 1: 'xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xxxx xxxx.xx

In [5]:
from zeroshot_topics import ZeroShotTopicFinder
import nltk
nltk.download('wordnet')
zsmodel = ZeroShotTopicFinder()


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bishw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
zsmodel.find_topic(cluster_map_1.get(14))

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Telephone', 'Inclination']

In [11]:
for k,v in cluster_map_1.items():
    try:
        print(zsmodel.find_topic(cluster_map_1.get(k)))
    except Exception as e:
        print(f"Not available for {k}")
        continue

Not available for 0
Not available for 1


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Condition', 'Activity']


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Acknowledgment', 'Aid']


...

['Positive Identification', 'Sanction']


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Indebtedness', 'Liabilities']


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Series', 'Time Period']


# Comparing with keybert

In [12]:
from keybert import KeyBERT

In [13]:
kw_model = KeyBERT()
for k,v in cluster_map_1.items():
    try:
        print(kw_model.extract_keywords(cluster_map_1.get(k), keyphrase_ngram_range=(1, 2), stop_words=None))
    except Exception as e:
        print(f"Not available for {k}")
        continue
    


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


[]


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


[('xxxx18', 0.3262), ('xxxx xxxxxxxxxxxx', 0.3201), ('ok xxxxxx', 0.3144), ('xxxx5', 0.3144), ('already xxxx', 0.311)]


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


...

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


[('card citibank', 0.5356), ('card number', 0.529), ('mastercard store', 0.4829), ('card mastercard', 0.4815), ('express credit', 0.4777)]


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


[('validation debt', 0.5678), ('validity debt', 0.566), ('debt validation', 0.5574), ('debt request', 0.5471), ('verified debt', 0.5413)]
[('payments monthly', 0.5592), ('monthly payments', 0.5585), ('monthly payment', 0.5369), ('time monthly', 0.5203), ('payments making', 0.516)]


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
